In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('E:/Blackoffer/Data.csv', encoding="utf-8")
df.head()

,text
0,How artificial intelligence can boost your pro...
1,How does AI help to monitor Retail Shelf watch...
2,How are genetic sequencing maps affected by de...
3,AI and its impact on the Fashion Industry - Bl...
4,Benefits of Big Data in Different fields - Bla...


In [3]:
# removing the punctuation and making lower case
import string 
def punctuation_remove(str):
    #fist two use for the replace the data
    return str.translate(str.maketrans('\xa0',' ',string.punctuation))
    
    

In [4]:
#create a copy of data and calling function
# here it can give chainedcopysetting error so you can supress using below line or use loc function to copy the data
pd.options.mode.chained_assignment = None 
data=df.copy()
for i in range(0,170):
    data['text'][i] = punctuation_remove(data['text'][i]).lower()
    data['text'][i] = ''.join([i for i in  data['text'][i] if not i.isdigit()])

In [5]:
data['text'][169]

'impacts of covid  on food products  blackcoffer insightssome vendors fruit and vegetable sellers began venturing out after a few days without explicit permission and immediately faced police harassment after a few weeks the government eased restrictions and essential vendors were being permitted to vend due in large part to the advocacy of vendor organizations and activist networks however the cost of doing business as well as the risk has gone up significantly with vendors not having access to wholesale markets and suppliers and having to spend more on travel costs due to travel restrictions in place in the city also with the lockdown still partially in place the number of buyers has gone down and so have earnings due to the harsh summer heat perishable fruits and vegetables also have a reduced shelf life so vendors are unable to capitalize on whatever produce they do havethe state has recently announced a stimulus package of inr  crore for nearly  lakh vendors acknowledging the grav

### tokenize Data

In [6]:
# tokenizing the word using nltk
from nltk.tokenize import word_tokenize
clean_data=data.copy()

In [7]:
for i in range(0, len(clean_data['text'])):
    clean_data['text'][i] = word_tokenize(clean_data['text'][i])

In [8]:
clean_data

,text
0,"[how, artificial, intelligence, can, boost, yo..."
1,"[how, does, ai, help, to, monitor, retail, she..."
2,"[how, are, genetic, sequencing, maps, affected..."
3,"[ai, and, its, impact, on, the, fashion, indus..."
4,"[benefits, of, big, data, in, different, field..."
...,...
165,"[how, artificial, intelligence, can, deliver, ..."
166,"[role, of, big, data, in, academia, blackcoffe..."
167,"[what, is, data, exfiltration, blackcoffer, in..."
168,"[methods, for, sales, forecasting, in, retail,..."


## removing stopwords

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [10]:
clean_data['text']=clean_data['text'].apply(lambda x: [item for item in x if item not in stop])

In [11]:
len(clean_data['text'][166])


810

In [12]:
clean_data['text']

0      [artificial, intelligence, boost, productivity...
1      [ai, help, monitor, retail, shelf, watches, bl...
2      [genetic, sequencing, maps, affected, deep, le...
3      [ai, impact, fashion, industry, blackcoffer, i...
4      [benefits, big, data, different, fields, black...
                             ...                        
165    [artificial, intelligence, deliver, real, valu...
166    [role, big, data, academia, blackcoffer, insig...
167    [data, exfiltration, blackcoffer, insightsif, ...
168    [methods, sales, forecasting, retail, blackcof...
169    [impacts, covid, food, products, blackcoffer, ...
Name: text, Length: 170, dtype: object

### lemmantization 

In [13]:
import nltk
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

cleans_data=clean_data.copy()

def lemmatize_text(text):
    return "".join([lemmatizer.lemmatize(w) for w in text]) 

for i in range(0, 169):
    for j in range(0,len(clean_data['text'][i])-1):
        cleans_data['text'][i][j-1]=lemmatize_text(clean_data['text'][i][j-1])

In [14]:
(cleans_data)

,text
0,"[artificial, intelligence, boost, productivity..."
1,"[ai, help, monitor, retail, shelf, watches, bl..."
2,"[genetic, sequencing, maps, affected, deep, le..."
3,"[ai, impact, fashion, industry, blackcoffer, i..."
4,"[benefits, big, data, different, fields, black..."
...,...
165,"[artificial, intelligence, deliver, real, valu..."
166,"[role, big, data, academia, blackcoffer, insig..."
167,"[data, exfiltration, blackcoffer, insightsif, ..."
168,"[methods, sales, forecasting, retail, blackcof..."


### creating positive negative and polarity score

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [16]:
def sentiment_neg_analyze(text):
    score = SentimentIntensityAnalyzer().polarity_scores(text)
    return score['neg']
def sentiment_pos_analyze(text):
    score = SentimentIntensityAnalyzer().polarity_scores(text)
    return score['pos']
def sentiment_neu_analyze(text):
    score = SentimentIntensityAnalyzer().polarity_scores(text)
    return score['neu']
cleans_data['neg']=0
cleans_data['pos']=0
cleans_data['neu']=0

In [20]:
#most time taker function
for i in range(0,169):
    neg=0
    pos=0
    neu=0
    for j in range(0,len(clean_data['text'][i])):
         neg += sentiment_neg_analyze(cleans_data['text'][i][j])
         pos += sentiment_pos_analyze(cleans_data['text'][i][j])
         neu += sentiment_neu_analyze(cleans_data['text'][i][j])
    cleans_data['neg'][i]=neg
    cleans_data['pos'][i]=pos
    cleans_data['neu'][i]=neu


In [21]:
cleans_data.head()

,text,neg,pos,neu
0,"[artificial, intelligence, boost, productivity...",5,30,327
1,"[ai, help, monitor, retail, shelf, watches, bl...",11,34,331
2,"[genetic, sequencing, maps, affected, deep, le...",7,46,622
3,"[ai, impact, fashion, industry, blackcoffer, i...",12,85,901
4,"[benefits, big, data, different, fields, black...",15,69,660


In [26]:
#creating two columns float type for polarity
cleans_data['polarity']=0
cleans_data['subjective']=0
cleans_data['polarity']=pd.to_numeric(cleans_data["polarity"], downcast="float")
cleans_data['subjective']=pd.to_numeric(cleans_data["subjective"], downcast="float")

### formula for the polarity and subjective

In [27]:
for i in range(0,169):
    cleans_data['polarity'][i]=(cleans_data['pos'][i]-cleans_data['neg'][i])/((cleans_data['pos'][i]+cleans_data['neg'][i])+0.000001)
for i in range(0,169):
    cleans_data['subjective'][i]=(cleans_data['pos'][i] + cleans_data['neg'][i])/(len(cleans_data['text'][i]) + 0.000001)

In [28]:
cleans_data

,text,neg,pos,neu,polarity,subjective
0,"[artificial, intelligence, boost, productivity...",5,30,327,0.714286,0.096154
1,"[ai, help, monitor, retail, shelf, watches, bl...",11,34,331,0.511111,0.118421
2,"[genetic, sequencing, maps, affected, deep, le...",7,46,622,0.735849,0.076369
3,"[ai, impact, fashion, industry, blackcoffer, i...",12,85,901,0.752577,0.094175
4,"[benefits, big, data, different, fields, black...",15,69,660,0.642857,0.111406
...,...,...,...,...,...,...
165,"[artificial, intelligence, deliver, real, valu...",14,84,733,0.714286,0.116390
166,"[role, big, data, academia, blackcoffer, insig...",14,51,730,0.569231,0.080247
167,"[data, exfiltration, blackcoffer, insightsif, ...",6,7,245,0.076923,0.050193
168,"[methods, sales, forecasting, retail, blackcof...",17,33,382,0.320000,0.113122


In [30]:
cleaned_data=cleans_data.copy()

In [32]:
cleaned_data.insert(0,'Text',df['text'])

In [61]:
cleaned_data['average_sentence_length']=0
cleaned_data['syllabe_word']=0
cleaned_data['fog_index']=0
cleaned_data['average_sentence_length']=pd.to_numeric(cleaned_data["average_sentence_length"], downcast="float")
cleaned_data['syllabe_word']=pd.to_numeric(cleaned_data["syllabe_word"], downcast="float")
cleaned_data['fog_index']=pd.to_numeric(cleaned_data["fog_index"], downcast="float")

###  Word Count and sentence count for the fog index

In [62]:
import nltk
from nltk.tokenize import sent_tokenize

def sentence_coutn(sentences):
    number_of_sentences = sent_tokenize(sentences)
    return len(number_of_sentences)
def word_count(c):
    return len(c.split())
    

In [63]:
for i in range(0,169):
    cleaned_data['average_sentence_length'][i]=(word_count(cleaned_data['Text'][i]))/(sentence_coutn(cleaned_data['Text'][i]))

In [64]:
cleaned_data['average_sentence_length']

0      21.281250
1      29.809525
2      29.184210
3      43.487179
4      36.411766
         ...    
165    27.830189
166    30.617022
167    22.571428
168    31.727272
169     0.000000
Name: average_sentence_length, Length: 170, dtype: float32